<a href="https://colab.research.google.com/github/202248SD/5DayGitChallenge/blob/main/ClusterData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/犬１-20240617T025624Z-001.zip
!unzip /content/犬２-20240617T025639Z-001.zip

In [6]:
!rm /content/犬１-20240617T025624Z-001.zip
!rm /content/犬２-20240617T025639Z-001.zip

In [141]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader, Dataset
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from PIL import Image
import os
import torchvision.models as models
import torch.nn as nn
from glob import glob
import cv2 as cv
import shutil

In [ ]:
%pip install ultralytics

In [4]:
from ultralytics import YOLO
DogDetector = YOLO("yolov8l.pt")

100%|██████████| 83.7M/83.7M [00:00<00:00, 171MB/s]


In [8]:
DogTypeNet = models.resnet50(pretrained=True)
num_ftrs = DogTypeNet.fc.in_features
DogTypeNet.fc = torch.nn.Linear(num_ftrs, 120)
DogTypeNet.load_state_dict(torch.load('/content/model (2).pth', map_location=torch.device('cpu')))
idx_to_class = {0: 'Chihuahua', 1: 'Japanese_spaniel', 2: 'Maltese_dog', 3: 'Pekinese', 4: 'Shih-Tzu', 5: 'Blenheim_spaniel', 6: 'papillon', 7: 'toy_terrier', 8: 'Rhodesian_ridgeback', 9: 'Afghan_hound', 10: 'basset', 11: 'beagle', 12: 'bloodhound', 13: 'bluetick', 14: 'black-and-tan_coonhound', 15: 'Walker_hound', 16: 'English_foxhound', 17: 'redbone', 18: 'borzoi', 19: 'Irish_wolfhound', 20: 'Italian_greyhound', 21: 'whippet', 22: 'Ibizan_hound', 23: 'Norwegian_elkhound', 24: 'otterhound', 25: 'Saluki', 26: 'Scottish_deerhound', 27: 'Weimaraner', 28: 'Staffordshire_bullterrier', 29: 'American_Staffordshire_terrier', 30: 'Bedlington_terrier', 31: 'Border_terrier', 32: 'Kerry_blue_terrier', 33: 'Irish_terrier', 34: 'Norfolk_terrier', 35: 'Norwich_terrier', 36: 'Yorkshire_terrier', 37: 'wire-haired_fox_terrier', 38: 'Lakeland_terrier', 39: 'Sealyham_terrier', 40: 'Airedale', 41: 'cairn', 42: 'Australian_terrier', 43: 'Dandie_Dinmont', 44: 'Boston_bull', 45: 'miniature_schnauzer', 46: 'giant_schnauzer', 47: 'standard_schnauzer', 48: 'Scotch_terrier', 49: 'Tibetan_terrier', 50: 'silky_terrier', 51: 'soft-coated_wheaten_terrier', 52: 'West_Highland_white_terrier', 53: 'Lhasa', 54: 'flat-coated_retriever', 55: 'curly-coated_retriever', 56: 'golden_retriever', 57: 'Labrador_retriever', 58: 'Chesapeake_Bay_retriever', 59: 'German_short-haired_pointer', 60: 'vizsla', 61: 'English_setter', 62: 'Irish_setter', 63: 'Gordon_setter', 64: 'Brittany_spaniel', 65: 'clumber', 66: 'English_springer', 67: 'Welsh_springer_spaniel', 68: 'cocker_spaniel', 69: 'Sussex_spaniel', 70: 'Irish_water_spaniel', 71: 'kuvasz', 72: 'schipperke', 73: 'groenendael', 74: 'malinois', 75: 'briard', 76: 'kelpie', 77: 'komondor', 78: 'Old_English_sheepdog', 79: 'Shetland_sheepdog', 80: 'collie', 81: 'Border_collie', 82: 'Bouvier_des_Flandres', 83: 'Rottweiler', 84: 'German_shepherd', 85: 'Doberman', 86: 'miniature_pinscher', 87: 'Greater_Swiss_Mountain_dog', 88: 'Bernese_mountain_dog', 89: 'Appenzeller', 90: 'EntleBucher', 91: 'boxer', 92: 'bull_mastiff', 93: 'Tibetan_mastiff', 94: 'French_bulldog', 95: 'Great_Dane', 96: 'Saint_Bernard', 97: 'Eskimo_dog', 98: 'malamute', 99: 'Siberian_husky', 100: 'affenpinscher', 101: 'basenji', 102: 'pug', 103: 'Leonberg', 104: 'Newfoundland', 105: 'Great_Pyrenees', 106: 'Samoyed', 107: 'Pomeranian', 108: 'chow', 109: 'keeshond', 110: 'Brabancon_griffon', 111: 'Pembroke', 112: 'Cardigan', 113: 'toy_poodle', 114: 'miniature_poodle', 115: 'standard_poodle', 116: 'Mexican_hairless', 117: 'dingo', 118: 'dhole', 119: 'African_hunting_dog'}

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 125MB/s]


In [181]:
model = DogTypeNet

model.eval()

modules = list(model.children())[:-1]
feature_extractor = nn.Sequential(*modules)

In [9]:
import numpy as np
from glob import glob
inu_1 = np.array(glob("犬１/*"))
inu_2 = np.array(glob("犬２/*"))

In [11]:
print(inu_1, inu_2)

['犬１/IMG_2392.JPG' '犬１/IMG_7842.JPG' '犬１/IMG_9769.JPG' '犬１/IMG_2516.JPG' '犬１/IMG_8840.JPG' '犬１/IMG_0606.JPG' '犬１/IMG_7258.JPG' '犬１/IMG_2387.JPG' '犬１/IMG_6122.JPG' '犬１/IMG_1796.JPG' '犬１/IMG_8698.JPG' '犬１/IMG_0614.JPG' '犬１/IMG_5230.JPG' '犬１/IMG_5789.JPG' '犬１/IMG_2519.JPG' '犬１/IMG_3488.JPG' '犬１/IMG_8699.JPG'
 '犬１/IMG_5792.JPG' '犬１/IMG_2724.JPG' '犬１/IMG_8217.JPG'] ['犬２/IMG_6879.JPG' '犬２/IMG_6910.JPG' '犬２/IMG_6912.JPG' '犬２/IMG_6864.JPG' '犬２/IMG_6899.JPG' '犬２/IMG_6903.JPG' '犬２/IMG_6870.JPG' '犬２/IMG_6897.JPG' '犬２/IMG_6882.JPG' '犬２/IMG_6869.JPG' '犬２/IMG_6891.JPG' '犬２/IMG_6884.JPG' '犬２/IMG_6868.JPG' '犬２/IMG_6909.JPG' '犬２/IMG_6867.JPG' '犬２/IMG_6860.JPG' '犬２/IMG_6876.JPG'
 '犬２/IMG_6859.JPG' '犬２/IMG_6878.JPG' '犬２/IMG_6871.JPG' '犬２/IMG_6895.JPG' '犬２/IMG_6888.JPG' '犬２/IMG_6875.JPG' '犬２/IMG_6873.JPG' '犬２/IMG_6902.JPG' '犬２/IMG_6893.JPG' '犬２/IMG_6898.JPG' '犬２/IMG_6907.JPG' '犬２/IMG_6862.JPG' '犬２/IMG_6877.JPG' '犬２/IMG_6896.JPG' '犬２/IMG_6901.JPG' '犬２/IMG_6911.JPG' '犬２/IMG_6874.JPG'
 '犬２/IMG_6880.JPG' '犬２/

In [21]:
def process_image(img_path, box_coords=[]):
  preprocess = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

  image = Image.open(img_path).convert('RGB')

  if len(box_coords) != 0:
    image = image.crop(box_coords)

  image = preprocess(image)

  image = image.unsqueeze(0)

  return image

In [37]:
def predict(model, image, idx_to_label):
    model.eval()

    with torch.no_grad():

        output = model(image)

        _, predicted_idx = torch.max(output, 1)

        predicted_label = idx_to_label[predicted_idx.item()]

    return predicted_label

In [23]:
def format_label(label):
  if 'poodle' in label:
    label = 'Poodle'
  elif 'terrier' in label:
    label = 'Terrier'
  else:
    label = label[0].upper()+label[1:]
  return label

In [182]:
def extract_features(image):
  with torch.no_grad():
      features = feature_extractor(image)
  return features.squeeze()

In [26]:
def crop_save(input_path, crop_coords, output_path):
    with Image.open(input_path) as img:
        cropped_img = img.crop(crop_coords)
        cropped_img.save(output_path)

In [183]:
def get_dogs(img_paths):
  dataset_data = []
  print(len(img_paths))
  for img_path in img_paths:
    results = DogDetector(img_path, conf=0.3, iou=0.3)

    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()

    img = cv.imread(img_path)
    image_data = [] # [dog_img_path, label, ftrs]
    dog_idx = 1

    dir_path = 'Dogs/' + img_path[:len(img_path)-4]
    os.makedirs(dir_path, exist_ok=True)

    for box, cls in zip(boxes, classes):
      if cls != 16:
        continue
      else:
        path = dir_path + f'/Dog{dog_idx}.JPG'

        box_coords = [int(i) for i in box]

        crop_save(img_path, box_coords, path)

        box_img = process_image(img_path, box_coords)

        features = extract_features(box_img).numpy()

        label = predict(DogTypeNet, box_img, idx_to_class)
        label = format_label(label)

        dog_data = [path, label, features]
        image_data.append(dog_data)
        dog_idx += 1

    dataset_data.append(image_data)
  return dataset_data

In [184]:
dataset1_data = get_dogs(inu_1)
dataset2_data = get_dogs(inu_2)

20

image 1/1 /content/犬１/IMG_2392.JPG: 640x640 1 dog, 3063.6ms
Speed: 4.6ms preprocess, 3063.6ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/犬１/IMG_7842.JPG: 480x640 1 person, 1 dog, 2 suitcases, 2 chairs, 1 dining table, 2262.9ms
Speed: 3.8ms preprocess, 2262.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/犬１/IMG_9769.JPG: 640x480 1 dog, 1 couch, 3333.3ms
Speed: 3.8ms preprocess, 3333.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/犬１/IMG_2516.JPG: 640x480 1 person, 1 dog, 1 laptop, 1 microwave, 1 refrigerator, 2224.1ms
Speed: 3.6ms preprocess, 2224.1ms inference, 5.1ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/犬１/IMG_8840.JPG: 640x480 1 dog, 2244.9ms
Speed: 3.6ms preprocess, 2244.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/犬１/IMG_0606.JPG: 480x640 1 person, 2 dogs, 1 tie, 2196.8ms
Speed: 3.6m

In [123]:
print(len(dataset1_data[5]))
print(dataset1_data[5])
print(dataset1_data[5][0])

2
[['Dogs/犬１/IMG_0606/Dog1.JPG', 'Papillon', array([          0,           0,           0, ...,     0.62408,    0.076767,           0], dtype=float32)], ['Dogs/犬１/IMG_0606/Dog2.JPG', 'Chihuahua', array([  0.0071979,           0,           0, ...,     0.21085,     0.72435,           0], dtype=float32)]]
['Dogs/犬１/IMG_0606/Dog1.JPG', 'Papillon', array([          0,           0,           0, ...,     0.62408,    0.076767,           0], dtype=float32)]


In [185]:
def cluster_images(dataset_data, n_clusters):
  features = [] # [img_path, ftrs1, ftrs2, ...]

  for image_data in dataset_data:
    for dog_data in image_data:
      if len(dog_data) == 0:
        continue
      features.append(dog_data[2])

  features = np.array(features)
  pca = PCA(n_components=16)
  pca.fit(features)
  pca_features = pca.transform(features)

  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  kmeans.fit(pca_features)
  idx = 0
  for dogs_data in dataset_data:
    for dog_data in dogs_data:
      if len(dog_data) == 0:
        continue
      dog_data.append(kmeans.labels_[idx])
      idx += 1

  return dataset_data

In [186]:
dataset1_data = cluster_images(dataset1_data, 2)
dataset2_data = cluster_images(dataset2_data, 4)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [188]:
import shutil
import os

dir_to_delete = '/content/Dog2'

# Delete directory and its contents
shutil.rmtree(dir_to_delete)

In [189]:
def seperate_clusters(dataset_data, dir_name, num_clusters):
  for i in range(num_clusters):
    dir_path = f'{dir_name}/Clusters/Cluster{i+1}'
    os.makedirs(dir_path, exist_ok=True)

  for dogs_data in dataset_data:
    for dog_data in dogs_data:
      if len(dog_data) == 0:
        continue
      img_path = dog_data[0]
      new_path = f'{dir_name}/Clusters/Cluster{dog_data[-1]+1}/{img_path[8:15]}{img_path[17:]}'
      shutil.copy(img_path, new_path)

In [190]:
seperate_clusters(dataset1_data, '/content/Dog1', 2)

In [191]:
seperate_clusters(dataset2_data, '/content/Dog2', 4)